In [ ]:


X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, test_size=0.2, random_state=43)

# Split the combined validation/test set into separate validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5, random_state=43)


def my_FNN(hidden_layers, hidden_layer_dim, activation,learning_rate, batch_size, epochs):
    # Define the model architecture
    model = tf.keras.Sequential()
    
    
    model.add(tf.keras.layers.Dense(hidden_layer_dim, activation=activation, input_shape=(X_train.shape[1],)))
    model.add(tf.keras.layers.Dropout(0.2))


    for i in range(hidden_layers-1):
            model.add(tf.keras.layers.Dense(hidden_layer_dim, activation=activation))
            model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
   
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

    # Train the model on the training set and validate on the validation set
    model.fit(X_train, y_train, batch_size,epochs, validation_data=(X_val, y_val))

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print('Test accuracy:', test_acc)
    return model 

my_FNN(12,1024,'relu',0.001,512,150)


In [ ]:

param_space = {
    "hidden_layers": [2, 3, 4],
    "hidden_layer_dim": [64, 128, 256],
    "activation": ["relu", "sigmoid"],
    "optimizer": ["adam", "sgd"],
    "learning_rate": [0.01, 0.001, 0.0001],
    "batch_size": [128, 256, 512],
    "epochs": [100, 150, 200],
}
def fitness(params,X_train, y_train, X_val, y_val):
    # Create the model
    model = KerasClassifier(my_FNN, **params)
    # Train the model on the training data
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=params["epochs"], batch_size=params["batch_size"])
    # Evaluate the model on the validation set
    score = model.score(X_val, y_val)
    # Return the accuracy as the fitness
    return score

def evaluate_individual(individual, X_train, y_train, X_val, y_val, X_test, y_test):
    print(type(individual))
    params = {
        "hidden_layers": individual[0],
        "hidden_layer_dim": individual[1],
        "activation": individual[2],
        "optimizer": individual[3],
        "learning_rate": individual[4],
        "batch_size": individual[5],
        "epochs": individual[6]
    }
    accuracy = fitness(params, X_train, y_train, X_val, y_val)
   
    return accuracy,

# Define the genetic operators
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def init_individual(toolbox):
    return (toolbox.hidden_layers(),
            toolbox.hidden_layer_dim(),
            toolbox.activation(),
            toolbox.optimizer(),
            toolbox.learning_rate(),
            toolbox.batch_size(),
            toolbox.epochs())



toolbox.register("hidden_layers", random.choice, param_space["hidden_layers"])
toolbox.register("hidden_layer_dim", random.choice, param_space["hidden_layer_dim"])
toolbox.register("activation", random.choice, param_space["activation"])
toolbox.register("optimizer", random.choice, param_space["optimizer"])
toolbox.register("learning_rate", random.choice, param_space["learning_rate"])
toolbox.register("batch_size", random.choice, param_space["batch_size"])
toolbox.register("epochs", random.choice, param_space["epochs"])

toolbox.register("individual", tools.initIterate, creator.Individual, 
                 functools.partial(init_individual, toolbox))


toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate_individual, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)
toolbox.register("mate", tools.cxUniform, indpb=0.1)

toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(param_space["optimizer"]) - 1, indpb=0.1)

toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=100)

NGEN = 10


best_fitness = -float('inf')
num_generations_without_improvement = 0
MAX_GENERATIONS_WITHOUT_IMPROVEMENT = 3
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.2)
    fits = list(toolbox.map(toolbox.evaluate, offspring))    
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = (fit,)
    
    population = toolbox.select(offspring, k=len(population))
    top1 = tools.selBest(population, k=1)
    current_fitness = top1[0].fitness.values[0]
    print("Generation %i, Top 1 accuracy %f" % (gen, current_fitness))
    
    if current_fitness > best_fitness:
        best_fitness = current_fitness
        num_generations_without_improvement = 0
    else:
        num_generations_without_improvement += 1
        if num_generations_without_improvement >= MAX_GENERATIONS_WITHOUT_IMPROVEMENT:
            print("Stopping evolution process because there have been no improvements in the last %d generations." % MAX_GENERATIONS_WITHOUT_IMPROVEMENT)
            break

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import uniform, randint

# Define the parameter space for hyperparameters
param_dist = {'C': uniform(0.1, 10),
              'gamma': uniform(0.1, 1),
              'kernel': ['rbf', 'linear', 'poly']}

# Create an SVM object
svc = SVC()

# Define the RandomizedSearchCV object with 10-fold cross-validation and 20 iterations
random_search = RandomizedSearchCV(estimator=svc,
                                   param_distributions=param_dist,
                                   n_iter=20,
                                   cv=10,
                                   n_jobs=-1)

# Fit the model to the data
random_search.fit(X_train, y_train)

# Print the best hyperparameters and accuracy score
print("Best hyperparameters: ", random_search.best_params_)
print("Best accuracy score: ", random_search.best_score_)


In [5]:

models = ['Logistic regression','Decision Tree', 'Randomn forrest','XGBoost']
accuracies = [91,90,91,91]

result_dict = {key:value for key,value in zip(models, accuracies)}
sorted_dict = dict(sorted(result_dict.items(), key=lambda x: x[1]))




{'Decision Tree': 90, 'Logistic regression': 91, 'Randomn forrest': 91, 'XGBoost': 91}


In [2]:
import pandas as pd
from sklearn.decomposition import PCA

# Create a sample DataFrame with two correlated columns
data = {'Column 1': [1, 2, 3, 4, 5], 'Column 2': [1, 2, 3, 4, 5]}
df = pd.DataFrame(data)

# Initialize PCA
pca = PCA(n_components=1)  # Set the desired number of components

# Fit PCA to the correlated columns
pca.fit(df[['Column 1', 'Column 2']])

# Transform the data using the fitted PCA
transformed_data = pca.transform(df[['Column 1', 'Column 2']])

# Create a new DataFrame with the transformed data
df_transformed = pd.DataFrame(transformed_data, columns=['PC1'])

print("Original DataFrame:")
print(df)
print("\nTransformed DataFrame:")
print(df_transformed)

ValueError: Shape of passed values is (5, 2), indices imply (5, 1)